In [1]:
import os

from langchain.llms import AI21
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import streamlit as st


from langchain.document_loaders import PyPDFLoader

import pandas as pd
import traceback              #

from dotenv import load_dotenv
import json

In [2]:
# 0ZhUmciGlmjOZ4oGsUesOAQhWrGtpYMJ

In [3]:
# load the enviroment variable first 

load_dotenv()

True

In [4]:
# ai21 API key 
key = os.getenv("AI21_API_KEY")

In [5]:
llm = AI21( temperature = 0.8)

In [6]:
llm

AI21(temperature=0.8, ai21_api_key=SecretStr('**********'))

In [7]:
# Output format of the project

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [8]:
# Template making
# Here, we are giving text, subject tone for hardness level of question, Number of question

template = """
Text: {text}
You are expert MCQ maker, Given the above text. It is your job to create a QUIZ of {number} of multiple choice questions for
{subject} students in {tone}.
Make sure the question are correct and not repeated questions.
Make sure the format of your response in RESPONSE_JSON below, use it as a guide.

Ensure to make {number} of MCQ's
###RESPONSE_JSON
{response_json}

"""

In [21]:
prompt_mcq = PromptTemplate(input_variables = ['text','number','subject','tone','response_json'],
                  template = template)

In [22]:
from langchain.chains import LLMChain

generator_chain = LLMChain(llm = llm, prompt = prompt_mcq, output_key = 'quiz')

In [23]:
# template for checking the given prompt response


template2 = """
you are exper in english grammar & writer. Given the MCQ question for {subject} students. \
you need to evaluate the complexity of the MCQ question according to the MCQ grammar and complete analysi of the quiz. Only use
max 50 words for complexity analysis. If the quiz is not as per the congitive and analytical abilties of the students, update the quiz questions which needs to be changed & change the tone which needs to be changed, which perfectly fits the student
abilities. 

QUIZ MCQ : 
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [24]:
quiz_evaluate_prompt = PromptTemplate(input_variables = ['subject','quiz'],
                                    template = template2)

In [25]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluate_prompt,output_key = 'review_llm_chain', verbose = True)

In [41]:
# Connecting both the chains by Seuential chain

eval_chain = SequentialChain(chains = [generator_chain, review_chain], 
                input_variables = ['text','number','subject','tone','response_json'],
                output_variables = ['quiz','review_llm_chain'])

In [34]:
with open('/Users/mohitverma/Documents/Dcoument/ML/Langchain/MCQ-genAI/sample.txt','r') as f:
    Text = f.read()

In [35]:
Text

'\nBones provide the structure for our bodies. The adult human skeleton is made up of 206 bones. These include the bones of the skull, spine (vertebrae), ribs, arms and legs. Bones are made of connective tissue reinforced with calcium and specialised bone cells. Most bones also contain bone marrow, where blood cells are made.\n\nBones work with muscles and joints to hold our body together and support freedom of movement. This is called the musculoskeletal system. The skeleton supports and shapes the body and protects delicate internal organs such as the brain, heart and lungs.\n\nBones contain most of our body’s calcium supply. The body is constantly building up and breaking down bone tissue as required. Healthy bone needs a balanced diet, regular weight-bearing exercise and the right levels of various hormones.\n\n\nThe human skeleton is made up of 206 bones, including bones of the:\n\nSkull – including the jaw bone\nSpine – cervical, thoracic and lumbar vertebrae, sacrum and tailbone

In [36]:
Text 
number = 4
subject = "AI"
Tone = 'Hard'
response_json = RESPONSE_JSON

In [45]:
# 

eval_chain({"text": Text,
            'number': number, 'subject':subject, 'tone' : Tone, 'response_json' : response_json} )



> Entering new LLMChain chain...
Prompt after formatting:

you are exper in english grammar & writer. Given the MCQ question for AI students. you need to evaluate the complexity of the MCQ question according to the MCQ grammar and complete analysi of the quiz. Only use
max 50 words for complexity analysis. If the quiz is not as per the congitive and analytical abilties of the students, update the quiz questions which needs to be changed & change the tone which needs to be changed, which perfectly fits the student
abilities. 

QUIZ MCQ : 
{
  "1": {
    "mcq": "How many bones are in the human body?",
    "options": {
      "a": "1",
      "b": "2",
      "c": "3",
      "d": "206"
    },
    "correct": "206"
  },
  "2": {
    "mcq": "What type of bone has a long, thin shape?",
    "options": {
      "a": "long",
      "b": "short",
      "c": "flat",
      "d": "irregular"
    },
    "correct": "long"
  },
  "3": {
    "mcq": "What are the layers of bone tissue?",
    "options": {
   

{'text': '\nBones provide the structure for our bodies. The adult human skeleton is made up of 206 bones. These include the bones of the skull, spine (vertebrae), ribs, arms and legs. Bones are made of connective tissue reinforced with calcium and specialised bone cells. Most bones also contain bone marrow, where blood cells are made.\n\nBones work with muscles and joints to hold our body together and support freedom of movement. This is called the musculoskeletal system. The skeleton supports and shapes the body and protects delicate internal organs such as the brain, heart and lungs.\n\nBones contain most of our body’s calcium supply. The body is constantly building up and breaking down bone tissue as required. Healthy bone needs a balanced diet, regular weight-bearing exercise and the right levels of various hormones.\n\n\nThe human skeleton is made up of 206 bones, including bones of the:\n\nSkull – including the jaw bone\nSpine – cervical, thoracic and lumbar vertebrae, sacrum and

In [46]:
# The output is in string, we need this json

quiz1 = eval_chain({"text": Text,
            'number': 5, 'subject':subject, 'tone' : 'Extremely hard', 'response_json' : response_json} )



> Entering new LLMChain chain...
Prompt after formatting:

you are exper in english grammar & writer. Given the MCQ question for AI students. you need to evaluate the complexity of the MCQ question according to the MCQ grammar and complete analysi of the quiz. Only use
max 50 words for complexity analysis. If the quiz is not as per the congitive and analytical abilties of the students, update the quiz questions which needs to be changed & change the tone which needs to be changed, which perfectly fits the student
abilities. 

QUIZ MCQ : 
{
  "1": {
    "mcq": "The human skeleton is made up of how many bones?",
    "options": {
      "a": "500",
      "b": "1000",
      "c": "206",
      "d": "100"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "What are the four different types of bone in the human body?",
    "options": {
      "a": "cartilage",
      "b": "flexible",
      "c": "long bone, short bone, flat bone, irregular bone",
      "d": "long bone, short bone, flat bone"
   

In [68]:
quiz = quiz1.get('quiz')


In [69]:
print(quiz.strip())

{
  "1": {
    "mcq": "The human skeleton is made up of how many bones?",
    "options": {
      "a": "500",
      "b": "1000",
      "c": "206",
      "d": "100"
    },
    "correct": "c"
  },
  "2": {
    "mcq": "What are the four different types of bone in the human body?",
    "options": {
      "a": "cartilage",
      "b": "flexible",
      "c": "long bone, short bone, flat bone, irregular bone",
      "d": "long bone, short bone, flat bone"
    },
    "correct": "c"
  },
  "3": {
    "mcq": "What are the layers of bone tissue?",
    "options": {
      "a": "bone marrow, dense tissue and spongy or cancellous tissue",
      "b": "bone marrow, dense tissue, spongy or cancellous tissue and periosteum",
      "c


In [70]:
quiz

'{\n  "1": {\n    "mcq": "The human skeleton is made up of how many bones?",\n    "options": {\n      "a": "500",\n      "b": "1000",\n      "c": "206",\n      "d": "100"\n    },\n    "correct": "c"\n  },\n  "2": {\n    "mcq": "What are the four different types of bone in the human body?",\n    "options": {\n      "a": "cartilage",\n      "b": "flexible",\n      "c": "long bone, short bone, flat bone, irregular bone",\n      "d": "long bone, short bone, flat bone"\n    },\n    "correct": "c"\n  },\n  "3": {\n    "mcq": "What are the layers of bone tissue?",\n    "options": {\n      "a": "bone marrow, dense tissue and spongy or cancellous tissue",\n      "b": "bone marrow, dense tissue, spongy or cancellous tissue and periosteum",\n      "c'

In [71]:


for key, val in quiz.items():
    print(key,val)

AttributeError: 'str' object has no attribute 'items'